In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
import joblib
import os
import re
from IPython.display import display
from tqdm.notebook import tqdm
import gc

In [2]:
print("Krok 0: Konfiguracja środowiska")

ARTIFACTS_DIR = "Artifacts_Polska"
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

LOC_FILE = 'lokalizacja.csv'
DATA_FILE_1 = 'saleflats_2024_dateAdded_polska.csv'
DATA_FILE_2 = 'saleflats_2024_newestDate_polska.csv'
OUTPUT_FILE = 'Location_Polska_Final_Tuned.csv'

MAX_TEXT_FEATURES = 5000

Krok 0: Konfiguracja środowiska


In [3]:
print("--- START: Budowa finalnego zbioru danych ---")

# --- Krok 1: Wczytanie pliku lokalizacja.csv ---
print("Wczytywanie i przygotowywanie pliku lokalizacja.csv...")
try:
    df_lokalizacja = pd.read_csv(
        'lokalizacja.csv', sep=',', header=None,
        names=['Id', 'ParentId', 'Name', 'AdditionalName', 'FullName'], dtype=str
    )
    df_lokalizacja['Id'] = pd.to_numeric(df_lokalizacja['Id'], errors='coerce')
    df_lokalizacja['ParentId'] = df_lokalizacja['ParentId'].replace('\\N', pd.NA)
    df_lokalizacja['ParentId'] = pd.to_numeric(df_lokalizacja['ParentId'], errors='coerce')
    df_lokalizacja.dropna(subset=['Id'], inplace=True)
    df_lokalizacja['Id'] = df_lokalizacja['Id'].astype(int)
    df_lokalizacja['ParentId'] = df_lokalizacja['ParentId'].astype('Int64')
    
    id_to_name = df_lokalizacja.set_index('Id')['Name'].to_dict()
    id_to_parent = df_lokalizacja.set_index('Id')['ParentId'].to_dict()
    city_to_districts = df_lokalizacja[df_lokalizacja['AdditionalName'].str.contains('Dzielnica|Osiedle', na=False)].groupby('ParentId')['Id'].apply(list).to_dict()
    district_to_streets = df_lokalizacja[df_lokalizacja['AdditionalName'] == 'Ulica'].groupby('ParentId')['Id'].apply(list).to_dict()
    print(f"Plik lokalizacja.csv wczytany.")
except FileNotFoundError:
    print("BŁĄD KRYTYCZNY: Nie znaleziono pliku lokalizacja.csv")
    exit()

# --- Krok 2: Ekstrakcja danych z plików ofert ---
try:
    first_row = pd.read_csv(DATA_FILE_1, sep=',', header=None, nrows=1, on_bad_lines='skip', encoding='utf-8')
    last_col_index = first_row.shape[1] - 1
    print(f"Automatycznie wykryto, że ostatnia kolumna ma indeks: {last_col_index}")
except:
    last_col_index = 52
    print(f"Nie udało się wykryć ostatniej kolumny, używam wartości awaryjnej: {last_col_index}")

COLS_TO_EXTRACT = {
    3: 'Title', 4: 'Description', 5: 'Area', 6: 'Price',
    9: 'NumberOfRooms', 16: 'BuiltYear',
    last_col_index: 'locationPath'
}

all_offers_df = []
for filepath in [DATA_FILE_1, DATA_FILE_2]:
    print(f"\nEkstrakcja danych z pliku: {filepath}...")
    try:
        df_chunk = pd.read_csv(
            filepath, sep=',', header=None, on_bad_lines='skip',
            usecols=list(COLS_TO_EXTRACT.keys()), low_memory=False, dtype=str, encoding='utf-8'
        )
        df_chunk.rename(columns=COLS_TO_EXTRACT, inplace=True)
        all_offers_df.append(df_chunk)
        print(f"Udało się wyekstrahować {len(df_chunk)} wierszy.")
    except Exception as e:
        print(f"Błąd podczas ekstrakcji z {filepath}: {e}")

if not all_offers_df:
    print("BŁĄD KRYTYCZNY: Nie udało się wczytać żadnych danych z plików ofert.")
    exit()

df = pd.concat(all_offers_df, ignore_index=True)
df.dropna(subset=['locationPath'], inplace=True)
df.drop_duplicates(subset=['Title', 'Description', 'locationPath'], keep='first', inplace=True)
df_original = df.copy()
print(f"\nPołączono i uzyskano {len(df)} unikalnych ofert.")

# --- Krok 3: Parsowanie ścieżki i tworzenie kolumn docelowych ---
print("Parsowanie ścieżek i tworzenie kolumn docelowych...")
path_cols = ['Wojewodztwo_ID', 'Powiat_ID', 'Gmina_ID', 'Miasto_ID', 'Dzielnica_ID', 'PodDzielnica_ID', 'Ulica_ID']
split_paths = df['locationPath'].str.split(',')
path_df = pd.DataFrame(split_paths.tolist(), index=df.index)
if path_df.shape[1] > 7: path_df = path_df.iloc[:, :7]
while path_df.shape[1] < 7: path_df[path_df.shape[1]] = None
path_df.columns = path_cols
path_df = path_df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
df = pd.concat([df.reset_index(drop=True), path_df.reset_index(drop=True)], axis=1)

df['target_district_id'] = np.where(df['PodDzielnica_ID'] != 0, df['PodDzielnica_ID'], df['Dzielnica_ID'])
df['target_city_id'] = df['Miasto_ID']
df['target_street_id'] = df['Ulica_ID']

df = df[df['target_city_id'] != 0].copy()
df.reset_index(drop=True, inplace=True)

if len(df) == 0:
    print("BŁĄD KRYTYCZNY: Po filtrowaniu nie pozostały żadne wiersze z określonym miastem.")
else:
    print(f"Liczba ogłoszeń z określonym miastem do treningu: {len(df)}")

--- START: Budowa finalnego zbioru danych ---
Wczytywanie i przygotowywanie pliku lokalizacja.csv...
Plik lokalizacja.csv wczytany.
Automatycznie wykryto, że ostatnia kolumna ma indeks: 52

Ekstrakcja danych z pliku: saleflats_2024_dateAdded_polska.csv...
Udało się wyekstrahować 793664 wierszy.

Ekstrakcja danych z pliku: saleflats_2024_newestDate_polska.csv...
Udało się wyekstrahować 755679 wierszy.

Połączono i uzyskano 773591 unikalnych ofert.
Parsowanie ścieżek i tworzenie kolumn docelowych...
Liczba ogłoszeń z określonym miastem do treningu: 709104


In [4]:
print("\nKrok 2: Inżynieria Cech (v1.3 - Finalna, Niezawodna)")
from sklearn.impute import SimpleImputer

# --- Przetwarzanie cech numerycznych ---
# ZMIANA: Używamy TYLKO 'Area' i 'Price', ponieważ inne indeksy są nieprawidłowe
num_features_final = ['Area', 'Price']
print(f"Używane cechy numeryczne: {num_features_final}")

for col in num_features_final:
    df[col] = pd.to_numeric(df[col], errors='coerce')

imputer = SimpleImputer(strategy='median')
df[num_features_final] = imputer.fit_transform(df[num_features_final])

scaler = StandardScaler()
X_num = scaler.fit_transform(df[num_features_final])
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, 'scaler_final.joblib'))

# --- Przetwarzanie cech tekstowych (z zachowaniem liczb) ---
df['text_features'] = df['Title'].fillna('') + ' ' + df['Description'].fillna('')

def clean_text_with_digits(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['text_features'] = df['text_features'].apply(clean_text_with_digits)

vectorizer = TfidfVectorizer(max_features=MAX_TEXT_FEATURES, ngram_range=(1, 1))

print("Rozpoczynam wektoryzację tekstu...")
X_text = vectorizer.fit_transform(df['text_features'])
joblib.dump(vectorizer, os.path.join(ARTIFACTS_DIR, 'vectorizer_final.joblib'))

print(f"Przygotowano cechy: {X_text.shape[1]} tekstowych i {X_num.shape[1]} numerycznych.")


Krok 2: Inżynieria Cech (v1.3 - Finalna, Niezawodna)
Używane cechy numeryczne: ['Area', 'Price']
Rozpoczynam wektoryzację tekstu...
Przygotowano cechy: 5000 tekstowych i 2 numerycznych.


In [5]:
print("\nKrok 3: Przygotowanie Celów (Target Variables) jako indeksy")

city_ids = sorted(df['target_city_id'].unique())
district_ids = sorted(df['target_district_id'].unique())
street_ids = sorted(df['target_street_id'].unique())

if 0 not in district_ids: district_ids.insert(0,0)
if 0 not in street_ids: street_ids.insert(0,0)

city_id_map = {id: i for i, id in enumerate(city_ids)}
district_id_map = {id: i for i, id in enumerate(district_ids)}
street_id_map = {id: i for i, id in enumerate(street_ids)}

joblib.dump(city_id_map, os.path.join(ARTIFACTS_DIR, 'city_id_map.joblib'))
joblib.dump(district_id_map, os.path.join(ARTIFACTS_DIR, 'district_id_map.joblib'))
joblib.dump(street_id_map, os.path.join(ARTIFACTS_DIR, 'street_id_map.joblib'))

inv_city_id_map = {i: id for id, i in city_id_map.items()}
inv_district_id_map = {i: id for id, i in district_id_map.items()}
inv_street_id_map = {i: id for id, i in street_id_map.items()}
joblib.dump(inv_city_id_map, os.path.join(ARTIFACTS_DIR, 'inv_city_id_map.joblib'))
joblib.dump(inv_district_id_map, os.path.join(ARTIFACTS_DIR, 'inv_district_id_map.joblib'))
joblib.dump(inv_street_id_map, os.path.join(ARTIFACTS_DIR, 'inv_street_id_map.joblib'))

y_city = df['target_city_id'].map(city_id_map).values
y_district = df['target_district_id'].map(district_id_map).values
y_street = df['target_street_id'].map(street_id_map).values

if np.isnan(y_city).any() or np.isnan(y_district).any() or np.isnan(y_street).any():
    print("BŁĄD: Wystąpiły wartości NaN po mapowaniu ID na indeksy.")
    exit()

print(f"Przygotowano zmienne docelowe. Liczba unikalnych klas:\nMiasta: {len(city_ids)}\nDzielnice: {len(district_ids)}\nUlice: {len(street_ids)}")


Krok 3: Przygotowanie Celów (Target Variables) jako indeksy
Przygotowano zmienne docelowe. Liczba unikalnych klas:
Miasta: 6933
Dzielnice: 1940
Ulice: 28177


In [6]:
print("\nKrok 4: Podział na Zbiory Danych")
X_text_train, X_text_val, X_num_train, X_num_val, y_city_train, y_city_val, y_district_train, y_district_val, y_street_train, y_street_val = train_test_split(
    X_text, X_num, y_city, y_district, y_street, test_size=0.2, random_state=42
)
print(f"Rozmiar zbioru treningowego: {X_text_train.shape[0]}\nRozmiar zbioru walidacyjnego: {X_text_val.shape[0]}")


Krok 4: Podział na Zbiory Danych
Rozmiar zbioru treningowego: 567283
Rozmiar zbioru walidacyjnego: 141821


In [7]:
print("\nKrok 5: Budowa i Kompilacja Modelu (v1.3 - Pełne Cechy + Dedykowana Głowa)")

# Wejścia
input_text = Input(shape=(MAX_TEXT_FEATURES,), name='text_input', sparse=True) 
# ZMIANA: Kształt wejścia numerycznego wraca do (None, 2)
input_num = Input(shape=(X_num_train.shape[1],), name='num_input') 

# --- Wspólna baza modelu ---
x1 = Dense(128, activation='relu')(input_text)
x1 = Dropout(0.4)(x1)

x2 = Dense(64, activation='relu')(input_num)
x2 = Dense(32, activation='relu')(x2)

combined = Concatenate()([x1, x2])
z_base = Dense(256, activation='relu')(combined)
z_base = Dropout(0.5)(z_base)

# --- Głowy dla Miasta i Dzielnicy ---
output_city = Dense(len(city_ids), activation='softmax', name='city_output')(z_base)
output_district = Dense(len(district_ids), activation='softmax', name='district_output')(z_base)

# --- Dedykowana, głębsza ścieżka tylko dla Ulicy ---
z_street = Dense(256, activation='relu')(z_base)
z_street = Dropout(0.4)(z_street)
output_street = Dense(len(street_ids), activation='softmax', name='street_output')(z_street)

model_final = Model(
    inputs=[input_text, input_num],
    outputs=[output_city, output_district, output_street]
)

# --- Kompilacja z wagami strat ---
optimizer_final = tf.keras.optimizers.Adam(learning_rate=0.0005)

model_final.compile(
    optimizer=optimizer_final,
    loss={
        'city_output': 'sparse_categorical_crossentropy',
        'district_output': 'sparse_categorical_crossentropy',
        'street_output': 'sparse_categorical_crossentropy'
    },
    loss_weights={
        'city_output': 1.0, 
        'district_output': 1.2,
        'street_output': 1.5
    },
    metrics={
        'city_output': 'accuracy',
        'district_output': 'accuracy',
        'street_output': 'accuracy'
    }
)

model_final.summary()


Krok 5: Budowa i Kompilacja Modelu (v1.3 - Pełne Cechy + Dedykowana Głowa)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 5000)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ num_input (InputLayer)        │ (None, 2)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │         640,128 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │             192 │ num_input[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 32)                │           2,080 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 160)               │               0 │ dropout[0][0],             │
│                               │                           │                 │ dense_2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 256)               │          41,216 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 256)               │               0 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 256)               │          65,792 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_2 (Dropout)           │ (None, 256)               │               0 │ dense_4[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_output (Dense)           │ (None, 6933)              │       1,781,781 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ district_output (Dense)       │ (None, 1940)              │         498,580 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ street_output (Dense)         │ (None, 28177)             │       7,241,489 │ dropout_2[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 10,271,258 (39.18 MB)

 Trainable params: 10,271,258 (39.18 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
print("\nKrok 6: Trening Modelu (Finalny Tuning)")

X_train_list = [X_text_train, X_num_train]
y_train_dict = {'city_output': y_city_train, 'district_output': y_district_train, 'street_output': y_street_train}
X_val_list = [X_text_val, X_num_val]
y_val_dict = {'city_output': y_city_val, 'district_output': y_district_val, 'street_output': y_street_val}

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-5, verbose=1)

history_final = model_final.fit(
    X_train_list,
    y_train_dict,
    validation_data=(X_val_list, y_val_dict),
    epochs=25,
    batch_size=256,
    callbacks=[early_stopping, reduce_lr]
)

model_final.save(os.path.join(ARTIFACTS_DIR, 'location_prediction_model_final.h5'))
print("Finalny, tuningowany model został wytrenowany i zapisany.")


Krok 6: Trening Modelu (Finalny Tuning)
Epoch 1/25
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 176s 78ms/step - city_output_accuracy: 0.1867 - city_output_loss: 4.8239 - district_output_accuracy: 0.4897 - district_output_loss: 3.9461 - loss: 16.0818 - street_output_accuracy: 0.6611 - street_output_loss: 4.3484 - val_city_output_accuracy: 0.5212 - val_city_output_loss: 2.9321 - val_district_output_accuracy: 0.5149 - val_district_output_loss: 2.6522 - val_loss: 11.5278 - val_street_output_accuracy: 0.6713 - val_street_output_loss: 3.6087 - learning_rate: 5.0000e-04
Epoch 2/25
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 156s 70ms/step - city_output_accuracy: 0.5276 - city_output_loss: 2.8503 - district_output_accuracy: 0.5113 - district_output_loss: 2.6403 - loss: 11.1699 - street_output_accuracy: 0.6701 - street_output_loss: 3.4341 - val_city_output_accuracy: 0.6145 - val_city_output_loss: 2.3466 - val_district_output_accuracy: 0.5393 - val_district_output_loss: 2.2263 - val_loss: 10.0150 - val_street_output_acc

Finalny, tuningowany model został wytrenowany i zapisany.


In [12]:
import gc
from tqdm.notebook import tqdm

print("\nKrok 7: Predykcja i Zastosowanie Logiki Hierarchicznej (Ręczna Pętla)")

n_samples = df.shape[0]
chunk_size = 50000

predicted_city_ids = []
predicted_district_ids = []
predicted_street_ids = []

print(f"Rozpoczynam predykcję na {n_samples} próbkach w kawałkach po {chunk_size}...")

for i in tqdm(range(0, n_samples, chunk_size)):
    chunk_text = X_text[i:i + chunk_size]
    chunk_num = X_num[i:i + chunk_size]
    chunk_list = [chunk_text, chunk_num]
    
    pred_city_chunk, pred_district_chunk, pred_street_chunk = model_final.predict(chunk_list, batch_size=512, verbose=0)
    
    for j in range(pred_city_chunk.shape[0]):
        city_pred_index = np.argmax(pred_city_chunk[j])
        city_id = inv_city_id_map.get(city_pred_index, 0)
        
        valid_districts_for_city = city_to_districts.get(city_id, [])
        # POPRAWKA: Używamy `pred_district_chunk`
        district_pred_index = np.argmax(pred_district_chunk[j]) 
        if valid_districts_for_city:
            district_mask = np.zeros_like(pred_district_chunk[j])
            valid_indices = [district_id_map.get(d_id) for d_id in valid_districts_for_city if district_id_map.get(d_id) is not None]
            if valid_indices:
                district_mask[valid_indices] = 1
                if np.sum(district_mask) > 0:
                    district_pred_index = np.argmax(pred_district_chunk[j] * district_mask)
        
        district_id = inv_district_id_map.get(district_pred_index, 0)
        
        valid_streets_for_district = district_to_streets.get(district_id, [])
        # POPRAWKA: Używamy `pred_street_chunk`
        street_pred_index = np.argmax(pred_street_chunk[j]) 
        if valid_streets_for_district:
            street_mask = np.zeros_like(pred_street_chunk[j])
            valid_indices = [street_id_map.get(s_id) for s_id in valid_streets_for_district if street_id_map.get(s_id) is not None]
            if valid_indices:
                street_mask[valid_indices] = 1
                if np.sum(street_mask) > 0:
                    street_pred_index = np.argmax(pred_street_chunk[j] * street_mask)

        street_id = inv_street_id_map.get(street_pred_index, 0)
        
        predicted_city_ids.append(city_id)
        predicted_district_ids.append(district_id)
        predicted_street_ids.append(street_id)
        
    del pred_city_chunk, pred_district_chunk, pred_street_chunk
    gc.collect()

df['predicted_city_id'] = predicted_city_ids
df['predicted_district_id'] = predicted_district_ids
df['predicted_street_id'] = predicted_street_ids

print("Zakończono predykcję z logiką hierarchiczną.")


Krok 7: Predykcja i Zastosowanie Logiki Hierarchicznej (Ręczna Pętla)
Rozpoczynam predykcję na 709104 próbkach w kawałkach po 50000...


  0%|          | 0/15 [00:00<?, ?it/s]

Zakończono predykcję z logiką hierarchiczną.


In [14]:
print("\nKrok 8: Generowanie i Zapis Wyników")

def create_loc_string(row):
    city = id_to_name.get(row['predicted_city_id'], "?")
    district = id_to_name.get(row['predicted_district_id'], "?")
    street = id_to_name.get(row['predicted_street_id'], "?")
    
    if row['predicted_district_id'] == 0: district = '?'
    if row['predicted_street_id'] == 0: street = '?'
        
    return f"{city} > {district} > {street}"

df['Predict_Loc'] = df.apply(create_loc_string, axis=1)

# Używamy df_original, które zapisaliśmy w komórce 3
df_to_merge = df[['Title', 'Description', 'locationPath', 'Predict_Loc']].copy()
df_final = pd.merge(df_original, df_to_merge, on=['Title', 'Description', 'locationPath'], how='left')
df_final.drop_duplicates(subset=['Title', 'Description', 'locationPath'], inplace=True)
df_final['Predict_Loc'].fillna('Brak predykcji', inplace=True)

df_final.to_csv(OUTPUT_FILE, index=False, sep=';', encoding='utf-8-sig')
print(f"Wyniki zostały zapisane do pliku: {OUTPUT_FILE}")


Krok 8: Generowanie i Zapis Wyników
Wyniki zostały zapisane do pliku: Location_Polska_Final_Tuned.csv


In [15]:
print("\nPrzykładowe 20 wierszy z wynikami:")

predicted_df = df_final[df_final['Predict_Loc'] != 'Brak predykcji']
if len(predicted_df) > 20:
    sample = predicted_df.sample(20)
else:
    sample = predicted_df.head(20)

display_cols = ['Title', 'locationPath', 'Predict_Loc']
sample_display = sample[display_cols]

def highlight_col(s):
    return ['background-color: #ffff99' if c == 'Predict_Loc' else '' for c in s.index]

styled_sample = sample_display.style.apply(highlight_col, axis=1)
display(styled_sample)


Przykładowe 20 wierszy z wynikami:


,Title,locationPath,Predict_Loc
27378,"Mieszkanie, Bydgoszcz, Kapuściska, 68 m²","10,0,0,374,0,0,0",Bydgoszcz > ? > ?
38361,"Sprzedam mieszkanie 56, 81m2","8,206,0,3909,0,0,0",Konin > ? > ?
399018,Seaport live in Gdynia Twoje nowe mieszkanie,"11,0,0,388,0,104414,455720",Gdynia > Grabówek > ?
557015,"Mieszkanie, Wrocław, Fabryczna, 50 m²","16,0,0,366,3270,72674,0",Wrocław > Fabryczna > ?
160103,3 pokojowe mieszkanie blisko toru Służewiec,"9,0,0,368,3284,91625,0",Warszawa > Ursynów > ?
572280,"Mieszkanie, Poznań, Naramowice, 73 m²","12,0,0,370,3261,81795,0",Poznań > ? > ?
197903,"Mieszkanie, Wrocław, Fabryczna, 80 m²","16,0,0,366,0,0,0",Wrocław > ? > ?
708702,Penthouse Z Windą Przy Parku Słowackiego - Chopina,"2,0,0,333,0,0,297485",Bielsko-biała > ? > ?
275684,"Mieszkanie 2 pokoje, 34,25 m2, Sady","1,0,0,381,0,0,0",Kielce > ? > ?
771369,Mieszkanie Warszawa,"9,0,0,368,3283,80732,0",Warszawa > Mokotów > ?
